In [ ]:
!pip install pandas pyarrow
import pandas as pd
!pip install transformers
!pip install SentencePiece
!pip install transformers
import os
import sys
import pandas as pd
from transformers import pipeline
from transformers import pipeline, set_seed
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel

In [2]:
# Pull Data from Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
abstracts = pd.read_parquet("/content/gdrive/MyDrive/w266_final_julia/df_abstract.parquet.gzip")
abstracts

Mounted at /content/gdrive


,file_key,abstract
0,0/links.,
1,a9012998,Excavations by Dr. Alan Simmons and his collea...
2,a9012376,This study will investigate organic matter deg...
3,a9012258,Professor Hull is engaged in an investigation ...
4,a9012685,"Because of recent scientific developments, phi..."
...,...,...
134611,3580.txt,"BCS-0123580 \tPI: Clifton, C. \tA three-day ..."
134612,a0123450,This proposal was submitted in response to the...
134613,a0123044,The R-Cubed program is providing scholarships ...
134614,a0123460,This proposal was submitted in response to the...


In [4]:
abs_sample = abstracts['abstract'][3]
first_sentence = abs_sample[0:205]
first_sentence

'Professor Hull is engaged in an investigation of two separateprojects:  the first concerns criteria for selection of taxonomicprinciples; the second concerns the processes by which scienceitself develops. '

In [5]:
abs_sample

'Professor Hull is engaged in an investigation of two separateprojects:  the first concerns criteria for selection of taxonomicprinciples; the second concerns the processes by which scienceitself develops.  While seemingly unrelated, Professor Hull isusing the first study as a case for exemplifying his argumentsconcerning the processes of scientific development.Although the distance between alternative principles of biologicalclassification and anything that might be considered "evidence" forthem is great, empirical considerations can be brought to bear onthe decision to choose one set of principles of classification overothers.  Professor Hull argues that the taxonomic principles thatresult in classifications that facilitate the discovery of naturalregularities are preferable to those that do not facilitate suchdiscoveries.  Dr Hull is examining the recent literature inevolutionary biology to see what sorts of classification aid eitherphylogenetic reconstruction or the discovery of re

In [9]:
# model_name = 't5-base'
# t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
# t5_model = TFT5ForConditionalGeneration.from_pretrained(model_name)
t5_model = TFT5ForConditionalGeneration.from_pretrained('t5-small')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
t5_model.summary()

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Model: "tft5_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60,506,624
Trainable params: 60,506,624
Non-trainable params: 0
_________________________________________________________________


In [13]:
ARTICLE = first_sentence
t5_input_text = "summarize: " + abs_sample
t5_inputs = t5_tokenizer([t5_input_text], return_tensors='tf')
t5_summary_ids = t5_model.generate(t5_inputs['input_ids'],
                                   num_beams=7, 
                                   no_repeat_ngram_size=2, 
                                   min_length=400, 
                                   max_length=600)

print([t5_tokenizer.decode(g, skip_special_tokens=True,
                           clean_up_tokenization_spaces=False)
       for g in t5_summary_ids])

['Falschsprachiges text: Professor Hull argues that taxonomic principles that facilitate the discovery of naturalregularities are preferable to those that do not facilitate suchdiscoveries. textuals will be used to support the contention that the use that scientistsmake of each other work is central to the ongoing process ofscience. he is examining the recent literature inevolutionary biology to see what sorts of classification aid eitherphylogenetic reconstruction or the discoveries of regularities in the enlightenment of scientific development.»»##160160;..? .A. thee.I.C a \'dis epaed resain the scientific ... d\'"doe if ve sentail " r.sir ??#?advanceds & l\'s.such u.S mrs?diassinthe neoodap ; i.d?en vs the science?»?Adopt trutruer142385010 com trgn there p?rienrmal?dpbli entr?n?f?""#"?(?&&#&?\' osisngan thaal f.rafs in b ] , g?s\'\'enthrilini?.;?;;s(id>?retailtails»An eng»" xioma\'r".")']


["professor Hull is examining literature inevolutionary biology to see what sorts of classification aid eitherphylogenetic reconstruction or the discovery of regularities . the results of these investigations will inturn be used to support the contention that the use that scientistsmake of each others work is central to the ongoing process ofscience. he is using the first study as a case for exemplifying his argumentsconcerning the processes by which scienceitself develops.the second is the process in which scientists make each other work. in  ed comcom d.p.s' sult in the 'the discovery i neossa's research will help the study to understand how how they can help resynapsia and the findings will supportthe argument that thatthe tyras in this study if it is to yy o c r ues of the scientific development of ? the theory that mcconnelly and other p ngs the science in their own v lincolns & x in and , of science ... and in some way thee gp in his own way - ; ). b wrisome and unrelated the research involving the method of scientific discovery and, in such encis entrina thyst the way it'd the other ins and outs, whose / based on the evidence feming tru..,the study will also be ’in then ....thee, theso other’s own science to make h ved in other' in scientific ingeads to help scientists – \xad england,’ and it will be"]


Professor Hull is engaged in an investigation of two separateprojects:  the first concerns criteria for selection of taxonomicprinciples; the second concerns the processes by which scienceitself develops.

In [ ]:
ARTICLE = first_sentence
t5_input_text = "generate text: " + abs_sample
t5_inputs = t5_tokenizer([t5_input_text], return_tensors='tf')

In [ ]:
t5_summary_ids = t5_model.generate(t5_inputs['input_ids'],
                                   num_beams=7, 
                                   no_repeat_ngram_size=2, 
                                   min_length=400, 
                                   max_length=600)

print([t5_tokenizer.decode(g, skip_special_tokens=True,
                           clean_up_tokenization_spaces=False)
       for g in t5_summary_ids])

['Falschsprachiges text: Professor Hull argues that taxonomic principles that facilitate the discovery of naturalregularities are preferable to those that do not facilitate suchdiscoveries. textuals will be used to support the contention that the use that scientistsmake of each other work is central to the ongoing process ofscience. he is examining the recent literature inevolutionary biology to see what sorts of classification aid eitherphylogenetic reconstruction or the discoveries of regularities in the enlightenment of scientific development.»»##160160;..? .A. thee.I.C a \'dis epaed resain the scientific ... d\'"doe if ve sentail " r.sir ??#?advanceds & l\'s.such u.S mrs?diassinthe neoodap ; i.d?en vs the science?»?Adopt trutruer142385010 com trgn there p?rienrmal?dpbli entr?n?f?""#"?(?&&#&?\' osisngan thaal f.rafs in b ] , g?s\'\'enthrilini?.;?;;s(id>?retailtails»An eng»" xioma\'r".")']


In [14]:
t5_model = TFT5ForConditionalGeneration.from_pretrained('t5-large')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-large')
t5_model.summary()

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Model: "tft5_for_conditional_generation_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  32899072  
                                                                 
 encoder (TFT5MainLayer)     multiple                  334939648 
                                                                 
 decoder (TFT5MainLayer)     multiple                  435627520 
                                                                 
Total params: 737,668,096
Trainable params: 737,668,096
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
ARTICLE = first_sentence
t5_input_text = "summarize : " + abs_sample
t5_inputs = t5_tokenizer([t5_input_text], return_tensors='tf')
t5_summary_ids = t5_model.generate(t5_inputs['input_ids'],
                                   num_beams=7, 
                                   no_repeat_ngram_size=2, 
                                   min_length=400, 
                                   max_length=600)

print([t5_tokenizer.decode(g, skip_special_tokens=True,
                           clean_up_tokenization_spaces=False)
       for g in t5_summary_ids])